In [2]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import fashion_mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline

In [3]:
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [4]:
from keras.datasets import mnist

In [5]:
from keras.datasets import cifar100

In [6]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [7]:
images=["Lenna.png"]

In [8]:
from keras.backend import clear_session

In [9]:
import numpy as np
import random
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

In [10]:
y2=Image.open("Lenna.png").resize([28,28]).convert("L")

In [11]:
np.random.seed(seed=1)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
tf.random.set_seed(0)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[6]
y2=Image.open("Lenna.png").resize([28,28]).convert("L")
for goal in images:
    #y2=Image.open(goal).resize([32,32])
    data2=fashion_mnist.load_data()[0][0]
    label2=fashion_mnist.load_data()[0][1]
    #label2=label2.reshape(len(label2))
    x_test=fashion_mnist.load_data()[1][0]
    y_test=fashion_mnist.load_data()[1][1]
    #y_test=y_test.reshape(len(y_test))
    #data3=np.zeros([len(data2),32,32,3])
    #for n in range(len(data2)):
        #data3[n]=Image.fromarray(data2[n].astype("uint8"))
    #data2=data3
    
    outputs=[]
    outputs4=[]
    outputs5=[]
    for c in range(len(convo)):
        outputs2=[]
        outputs3=[]
        for e in range(5):
            ensemble=[]
            for rot in degree:
                error_record=[]
                clear_session()
                unseen_score=np.zeros([10,9])
                AUC_score=np.zeros(10)
                for i in range(10):
                    x_train=data2[label2==i]
                    y_train=np.zeros([len(x_train),28,28,1])
                    for n in range(len(x_train)):
                        y_train[n]=np.array(y2.rotate(rot)).reshape(28,28,1)
                    #x_train=x_train/255
                    y_train=y_train
                    #x_test=data2[label2!=i]
                    #x_test=x_test/255
                    x_test=x_test.reshape(len(x_test),28,28,1)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    #(x_train, train), (x_test, y_test) = mnist.load_data()
                    image_height, image_width = 28, 28
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  image_height, image_width,1)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        y_train = y_train.reshape(y_train.shape[0],
                                                  28, 28,1)
                        input_shape = (image_height, image_width,1)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  1, image_height, image_width)
                        x_test = x_test.reshape(x_test.shape[0],1, image_height, image_width)
                        input_shape = (1, image_height, image_width)

                    # Min-Max Normalization (0. ~ 1. の範囲に値を収める)
                    x_train = x_train.astype('float32')
                    y_train = y_train.astype('float32')
                    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=e)
                    x_valid, x_valid2, y_valid, y_valid2 = train_test_split(x_valid, y_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    # 畳み込み演算を用いた AutoEncoder のネットワーク (Sequential API)
                    model = models.Sequential()
                    # 28 x 28 x 1
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(convo[c]):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(1, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',
                                  loss='mean_absolute_error')
                    # モデルの構造を確認する
                    fit_callbacks = [
                        callbacks.EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                mode='min')
                    ]
                    # モデルを学習させる
                    model.fit(x_train, y_train,
                              epochs=100,
                              batch_size=16,
                              shuffle=True,
                              validation_data=(x_valid, y_valid),callbacks=fit_callbacks)
                    result=model.predict(x_valid2)
                    result2=model.predict(x_test)
                    y_train=y_train.reshape(y_train.shape[0],784,1)
                    result=result.reshape(result.shape[0],784,1)
                    result2=result2.reshape(result2.shape[0],784,1)
                    loss=np.zeros(len(result))
                    for n in range(len(result)):
                        loss[n]=mean_absolute_error(y_train[0],result[n])
                    loss_outlier=np.zeros(len(result2))
                    for n in range(len(result2)):
                        loss_outlier[n]=mean_absolute_error(y_train[0],result2[n])
                    error_record.append(loss_outlier)
                    #y_one=np.ones(len(loss))
                    #y_outlier=np.zeros(len(loss_outlier))
                    #data=np.concatenate([loss,loss_outlier])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([loss,loss_outlier],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    AUC_score[i]=roc_auc_score(y_label,(-1)*loss_outlier)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(loss)*p)+1
                        #print(loss[loss.argsort()[-threshold]])
                        y_pred[loss_outlier<=loss[loss.argsort()[-threshold]]]=1
                        #loss_outlier[loss_outlier>loss[loss.argsort()[-threshold]]
                        unseen_score[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                #print(unseen_score)
                #outputs.append(unseen_score)
                outputs2.append(AUC_score)
                outputs3.append(unseen_score)
                ensemble.append(error_record)
            outputs5.append(ensemble)
            outputs.append(outputs2)
            outputs4.append(outputs3)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

Train on 4200 samples, validate on 1260 samples
Epoch 1/100
4200/4200 [==============================] - 6s 1ms/step - loss: 39.1382 - val_loss: 28.0584
Epoch 2/100
4200/4200 [==============================] - 3s 602us/step - loss: 24.3655 - val_loss: 23.4114
Epoch 3/100
4200/4200 [==============================] - 3s 621us/step - loss: 21.3432 - val_loss: 20.9379
Epoch 4/100
4200/4200 [==============================] - 3s 601us/step - loss: 19.4240 - val_loss: 19.3635
Epoch 5/100
4200/4200 [==============================] - 2s 580us/step - loss: 18.4058 - val_loss: 18.3815
Epoch 6/100
4200/4200 [==============================] - 2s 573us/step - loss: 17.4890 - val_loss: 17.7892
Epoch 7/100
4200/4200 [==============================] - 4s 1ms/step - loss: 16.6064 - val_loss: 16.5779
Epoch 8/100
4200/4200 [==============================] - 4s 1ms/step - loss: 15.9615 - val_loss: 16.9885
Epoch 9/100
4200/4200 [==============================] - 4s 910us/step - loss: 15.3473 - val_loss: 15.

4200/4200 [==============================] - 2s 500us/step - loss: 12.5571 - val_loss: 14.0405
Epoch 13/100
4200/4200 [==============================] - 2s 500us/step - loss: 12.2145 - val_loss: 12.3313
Epoch 14/100
4200/4200 [==============================] - 2s 500us/step - loss: 12.0504 - val_loss: 11.9301
Epoch 15/100
4200/4200 [==============================] - 2s 502us/step - loss: 11.7508 - val_loss: 12.2013
Epoch 16/100
4200/4200 [==============================] - 2s 501us/step - loss: 11.6598 - val_loss: 11.6465
Epoch 17/100
4200/4200 [==============================] - 2s 498us/step - loss: 11.4235 - val_loss: 13.0267
Epoch 18/100
4200/4200 [==============================] - 2s 496us/step - loss: 11.2928 - val_loss: 11.3620
Epoch 19/100
4200/4200 [==============================] - 2s 497us/step - loss: 11.2174 - val_loss: 11.1324
Epoch 20/100
4200/4200 [==============================] - 2s 502us/step - loss: 10.8563 - val_loss: 11.4591
Epoch 21/100
4200/4200 [=================

4200/4200 [==============================] - 2s 518us/step - loss: 11.7219 - val_loss: 11.9008
Epoch 23/100
4200/4200 [==============================] - 2s 515us/step - loss: 11.5698 - val_loss: 12.1213
Epoch 24/100
4200/4200 [==============================] - 2s 498us/step - loss: 11.5060 - val_loss: 11.5162
Epoch 25/100
4200/4200 [==============================] - 2s 498us/step - loss: 11.3605 - val_loss: 11.3507
Epoch 26/100
4200/4200 [==============================] - 2s 503us/step - loss: 11.3146 - val_loss: 11.9368
Epoch 27/100
4200/4200 [==============================] - 2s 496us/step - loss: 11.2371 - val_loss: 11.5120
Epoch 28/100
4200/4200 [==============================] - 2s 502us/step - loss: 11.1347 - val_loss: 11.3218
Epoch 29/100
4200/4200 [==============================] - 2s 495us/step - loss: 10.9924 - val_loss: 11.2418
Epoch 30/100
4200/4200 [==============================] - 2s 500us/step - loss: 10.9000 - val_loss: 10.9718
Epoch 31/100
4200/4200 [=================

4200/4200 [==============================] - 2s 443us/step - loss: 14.8785 - val_loss: 15.1166
Epoch 12/100
4200/4200 [==============================] - 2s 440us/step - loss: 14.5952 - val_loss: 15.7637
Epoch 13/100
4200/4200 [==============================] - 2s 443us/step - loss: 14.4327 - val_loss: 14.5405
Epoch 14/100
4200/4200 [==============================] - 2s 442us/step - loss: 14.1147 - val_loss: 13.7313
Epoch 15/100
4200/4200 [==============================] - 2s 444us/step - loss: 14.0182 - val_loss: 14.1690
Epoch 16/100
4200/4200 [==============================] - 2s 443us/step - loss: 13.6257 - val_loss: 13.6734
Epoch 17/100
4200/4200 [==============================] - 2s 444us/step - loss: 13.3945 - val_loss: 13.4140
Epoch 18/100
4200/4200 [==============================] - 2s 465us/step - loss: 13.2496 - val_loss: 13.1686
Epoch 19/100
4200/4200 [==============================] - 2s 465us/step - loss: 13.1856 - val_loss: 13.2007
Epoch 20/100
4200/4200 [=================

4200/4200 [==============================] - 2s 438us/step - loss: 17.0603 - val_loss: 15.9070
Epoch 6/100
4200/4200 [==============================] - 2s 445us/step - loss: 15.8571 - val_loss: 15.8751
Epoch 7/100
4200/4200 [==============================] - 2s 434us/step - loss: 15.2949 - val_loss: 16.5499
Epoch 8/100
4200/4200 [==============================] - 2s 462us/step - loss: 14.8191 - val_loss: 14.9558
Epoch 9/100
4200/4200 [==============================] - 2s 454us/step - loss: 14.5729 - val_loss: 13.9596
Epoch 10/100
4200/4200 [==============================] - 2s 456us/step - loss: 14.0791 - val_loss: 13.4198
Epoch 11/100
4200/4200 [==============================] - 2s 459us/step - loss: 13.5895 - val_loss: 13.1586
Epoch 12/100
4200/4200 [==============================] - 2s 456us/step - loss: 13.1895 - val_loss: 13.8046
Epoch 13/100
4200/4200 [==============================] - 2s 463us/step - loss: 13.2963 - val_loss: 13.8679
Epoch 14/100
4200/4200 [=====================

...

4200/4200 [==============================] - 2s 501us/step - loss: 10.3999 - val_loss: 10.4660
Epoch 51/100
4200/4200 [==============================] - 2s 498us/step - loss: 10.2969 - val_loss: 10.3627
Epoch 52/100
4200/4200 [==============================] - 2s 466us/step - loss: 10.3813 - val_loss: 11.0704
Epoch 53/100
4200/4200 [==============================] - 2s 479us/step - loss: 10.4058 - val_loss: 10.4416
Epoch 54/100
4200/4200 [==============================] - 2s 511us/step - loss: 10.2698 - val_loss: 10.8356
Epoch 55/100
4200/4200 [==============================] - 2s 511us/step - loss: 10.2568 - val_loss: 10.4690
Epoch 56/100
4200/4200 [==============================] - 2s 514us/step - loss: 10.2157 - val_loss: 10.5724
Train on 4200 samples, validate on 1260 samples
Epoch 1/100
4200/4200 [==============================] - 3s 608us/step - loss: 41.2119 - val_loss: 29.6598
Epoch 2/100
4200/4200 [==============================] - 2s 512us/step - loss: 27.8287 - val_loss: 26.

4200/4200 [==============================] - 2s 500us/step - loss: 20.9992 - val_loss: 21.0139
Epoch 6/100
4200/4200 [==============================] - 2s 501us/step - loss: 19.8954 - val_loss: 18.5779
Epoch 7/100
4200/4200 [==============================] - 2s 498us/step - loss: 18.9637 - val_loss: 17.8711
Epoch 8/100
4200/4200 [==============================] - 2s 506us/step - loss: 18.0855 - val_loss: 18.1076
Epoch 9/100
4200/4200 [==============================] - 2s 542us/step - loss: 17.7095 - val_loss: 17.8564
Epoch 10/100
4200/4200 [==============================] - 2s 518us/step - loss: 16.8630 - val_loss: 19.5725
Epoch 11/100
4200/4200 [==============================] - 2s 514us/step - loss: 16.6189 - val_loss: 16.1168
Epoch 12/100
4200/4200 [==============================] - 2s 509us/step - loss: 16.1153 - val_loss: 16.3112
Epoch 13/100
4200/4200 [==============================] - 2s 506us/step - loss: 15.7774 - val_loss: 15.3986
Epoch 14/100
4200/4200 [=====================

4200/4200 [==============================] - 2s 505us/step - loss: 10.4876 - val_loss: 10.8937
Epoch 82/100
4200/4200 [==============================] - 2s 507us/step - loss: 10.5806 - val_loss: 11.1222
Epoch 83/100
4200/4200 [==============================] - 2s 507us/step - loss: 10.5185 - val_loss: 10.8701
Epoch 84/100
4200/4200 [==============================] - 2s 505us/step - loss: 10.3789 - val_loss: 11.3987
Epoch 85/100
4200/4200 [==============================] - 2s 506us/step - loss: 10.5780 - val_loss: 11.0045
Epoch 86/100
4200/4200 [==============================] - 2s 506us/step - loss: 10.6212 - val_loss: 11.0477
Epoch 87/100
4200/4200 [==============================] - 2s 506us/step - loss: 10.5552 - val_loss: 10.7195
Epoch 88/100
4200/4200 [==============================] - 2s 502us/step - loss: 10.2955 - val_loss: 11.2678
Epoch 89/100
4200/4200 [==============================] - 2s 519us/step - loss: 10.2964 - val_loss: 10.9014
Epoch 90/100
4200/4200 [=================

4200/4200 [==============================] - 2s 506us/step - loss: 21.5779 - val_loss: 21.2013
Epoch 7/100
4200/4200 [==============================] - 2s 502us/step - loss: 20.7554 - val_loss: 20.5754
Epoch 8/100
4200/4200 [==============================] - 2s 507us/step - loss: 20.4486 - val_loss: 20.8817
Epoch 9/100
4200/4200 [==============================] - 2s 505us/step - loss: 19.9340 - val_loss: 19.3328
Epoch 10/100
4200/4200 [==============================] - 2s 509us/step - loss: 19.4587 - val_loss: 19.5219
Epoch 11/100
4200/4200 [==============================] - 2s 524us/step - loss: 19.1110 - val_loss: 19.1811
Epoch 12/100
4200/4200 [==============================] - 2s 539us/step - loss: 18.8074 - val_loss: 18.3172
Epoch 13/100
4200/4200 [==============================] - 2s 536us/step - loss: 18.4291 - val_loss: 20.2106
Epoch 14/100
4200/4200 [==============================] - 2s 538us/step - loss: 18.2690 - val_loss: 18.6179
Epoch 15/100
4200/4200 [====================

4200/4200 [==============================] - 2s 519us/step - loss: 13.4789 - val_loss: 13.2426
Epoch 24/100
4200/4200 [==============================] - 2s 508us/step - loss: 13.3836 - val_loss: 13.4739
Epoch 25/100
4200/4200 [==============================] - 2s 510us/step - loss: 13.2684 - val_loss: 13.5477
Epoch 26/100
4200/4200 [==============================] - 2s 512us/step - loss: 13.2733 - val_loss: 12.9920
Epoch 27/100
4200/4200 [==============================] - 2s 513us/step - loss: 13.3318 - val_loss: 13.1115
Epoch 28/100
4200/4200 [==============================] - 2s 510us/step - loss: 12.9494 - val_loss: 13.5777
Epoch 29/100
4200/4200 [==============================] - 2s 507us/step - loss: 12.9570 - val_loss: 13.3255
Epoch 30/100
4200/4200 [==============================] - 2s 514us/step - loss: 12.8278 - val_loss: 12.8770
Epoch 31/100
4200/4200 [==============================] - 2s 511us/step - loss: 12.7354 - val_loss: 13.5117
Epoch 32/100
4200/4200 [=================

In [12]:
outputs6=[]
for ensemble in outputs5:
    CEtest2=None
    value = 0
    for scores in ensemble:
        if type(CEtest2)==type(None):
            CEtest2=np.array(scores)
        else:
            print("OK")
            CEtest2=CEtest2+np.array(scores)
    AUC_ensemble=np.zeros(10)
    for i in range(10):
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        AUC_ensemble[i]=roc_auc_score(y_label,(-1)*CEtest2[i])
    outputs6.append(AUC_ensemble)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [13]:
np.array(outputs6).mean(axis=0)

array([0.95617938, 0.9917696 , 0.9388218 , 0.96634653, 0.94177902,
       0.94717736, 0.86018656, 0.98987613, 0.98272144, 0.98813091])

In [14]:
np.array(outputs6).std(axis=0)

array([0.00068128, 0.00036138, 0.00132716, 0.00052171, 0.00358254,
       0.00304269, 0.00073294, 0.00044329, 0.00076371, 0.00052   ])

In [15]:
AUC_score2=[]
for ensemble in outputs5:
    AUC_score=[]
    for scores in ensemble:
        AUC=np.zeros(10)
        for i in range(10):
            y_label=np.zeros(len(y_test))
            y_label[y_test==i]=1
            AUC[i]=roc_auc_score(y_label,(-1)*scores[i])
        AUC_score.append(AUC)
    AUC_score2.append(AUC_score)

In [16]:
np.array(AUC_score2).mean(axis=0)

array([[0.9582054 , 0.99161504, 0.93407823, 0.96402192, 0.93795613,
        0.95221437, 0.85736133, 0.98875478, 0.98240439, 0.98941911],
       [0.94185028, 0.9880062 , 0.94026446, 0.9538541 , 0.94300873,
        0.93387923, 0.85555176, 0.9889814 , 0.97308063, 0.98266378],
       [0.95359729, 0.99103016, 0.93073933, 0.95895468, 0.93625312,
        0.95485663, 0.85152419, 0.98780281, 0.96894348, 0.98701401],
       [0.95596527, 0.99185729, 0.93376156, 0.96211399, 0.93819279,
        0.93173519, 0.86272012, 0.98796513, 0.97571542, 0.98657862]])

In [17]:
np.array(AUC_score2).std(axis=0)

array([[0.00155044, 0.00027467, 0.00389312, 0.00150637, 0.00299386,
        0.00485335, 0.00179307, 0.00045113, 0.00153467, 0.00115301],
       [0.00280302, 0.00045101, 0.00079447, 0.00345202, 0.00560862,
        0.00257243, 0.00197692, 0.0006311 , 0.00285045, 0.00337683],
       [0.00098214, 0.00058862, 0.00194046, 0.00419895, 0.00304209,
        0.00263483, 0.00159805, 0.00040425, 0.0035037 , 0.00131025],
       [0.00181084, 0.00055503, 0.00407824, 0.00285717, 0.00430927,
        0.00361698, 0.00284274, 0.00058775, 0.00177915, 0.00105826]])

In [18]:
np.array(outputs6).mean(axis=0).mean()

0.9562988733333334

In [19]:
np.array(outputs6).std(axis=0)

array([0.00068128, 0.00036138, 0.00132716, 0.00052171, 0.00358254,
       0.00304269, 0.00073294, 0.00044329, 0.00076371, 0.00052   ])

In [20]:
outputs7=[]
outputs7.append(np.array(AUC_score2).mean(axis=0))
outputs7.append(np.array(AUC_score2).std(axis=0))
outputs7.append(np.array(outputs6).mean(axis=0))
outputs7.append(np.array(outputs6).std(axis=0))

In [21]:
outputs7

[array([[0.9582054 , 0.99161504, 0.93407823, 0.96402192, 0.93795613,
         0.95221437, 0.85736133, 0.98875478, 0.98240439, 0.98941911],
        [0.94185028, 0.9880062 , 0.94026446, 0.9538541 , 0.94300873,
         0.93387923, 0.85555176, 0.9889814 , 0.97308063, 0.98266378],
        [0.95359729, 0.99103016, 0.93073933, 0.95895468, 0.93625312,
         0.95485663, 0.85152419, 0.98780281, 0.96894348, 0.98701401],
        [0.95596527, 0.99185729, 0.93376156, 0.96211399, 0.93819279,
         0.93173519, 0.86272012, 0.98796513, 0.97571542, 0.98657862]]),
 array([[0.00155044, 0.00027467, 0.00389312, 0.00150637, 0.00299386,
         0.00485335, 0.00179307, 0.00045113, 0.00153467, 0.00115301],
        [0.00280302, 0.00045101, 0.00079447, 0.00345202, 0.00560862,
         0.00257243, 0.00197692, 0.0006311 , 0.00285045, 0.00337683],
        [0.00098214, 0.00058862, 0.00194046, 0.00419895, 0.00304209,
         0.00263483, 0.00159805, 0.00040425, 0.0035037 , 0.00131025],
        [0.00181084, 0.00

In [22]:
makefile(outputs7,"ITNensemble/fashionmnist.pkl")